# Practica Final

- Laura Quintero
- Lizeht Ormaza
- Vanessa Benites
- Mirele Borges

Para hablar com Alexa diga el código de la moneda a la que desea convertir. Por ejemplo: **Alexa me dice cuánto son 10 euros en BRL.**


In [1]:
import speech_recognition as sr

import pandas as pd

import nltk
from nltk.tokenize import word_tokenize

from nltk.corpus import cess_esp
from nltk.tag.hmm import HiddenMarkovModelTagger

from nltk.chunk.regexp import *

from datetime import datetime
import wikipedia

from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
from lat_lon_parser import parse
import requests
import pyttsx3
from deep_translator import GoogleTranslator

import openai

from word2number import w2n

import json

In [2]:
input_train = [
    [('Alexa', 'npcfs000'), ('dime', 'vmmp2s0'), ('cuánto', 'pt3ms000'), ('es', 'vmmp2s0'), ('cien', 'mccp00'), ('euros', 'ncmp000'), ('en', 'sps00'), ('USD', 'ncfs000')] 
]


In [3]:
hmm = HiddenMarkovModelTagger.train(input_train)

In [4]:
reglas_moneda = r'''
    Accion: {<vmmp2s0> <pt3ms000> | <vmmp2s0>}
    Cantidad: <Accion> {<mc.*>}
    Moneda: <sp.*> {<nc.*>}
    '''
parser = nltk.RegexpParser(reglas_moneda)

In [5]:
traductor_es = GoogleTranslator(source='en', target='es')
traductor_en = GoogleTranslator(source='es', target='en')

In [6]:
def tokenizar(_texto):
    
    return word_tokenize(_texto, 'spanish')

In [7]:
def parsear(_tokens):
    
    return parser.parse(_tokens)

In [8]:
def extrae_moneda(_tree): 

    
    data_moneda = ""
    for nodo in _tree:
        if type(nodo) == tuple:
          continue
        tipo = nodo.label()
        for elemento in nodo:
          if type(elemento) != tuple:
            continue
          palabra, categoria = elemento
          if tipo == 'Moneda':
            data_moneda=palabra
    
    return data_moneda

In [9]:

def extrae_cantidad(_tree):
    
    data_cantidad = ""
    for nodo in _tree:
        if type(nodo) == tuple:
          continue
        tipo = nodo.label()
        for elemento in nodo:
          if type(elemento) != tuple:
            continue
          palabra, categoria = elemento
          if tipo == 'Cantidad':
            data_cantidad= palabra
            
            traduccion = traductor_en.translate(data_cantidad)
            result_cantidad = w2n.word_to_num(traduccion)
            
    return result_cantidad

In [16]:
def convierte_moneda(_input_text, _hmm):
    
   
    testp = hmm.tag(tokenizar(_input_text))
    tree = parsear(testp)
    
    Moneda = extrae_moneda(tree)
    Cantidad = extrae_cantidad(tree)
    
    APIUrl = 'https://api.apilayer.com/fixer/convert?to='

    #Moneda = extrae_moneda(tree) + '&from=EUR'
    #Cantidad = '&amount=' + str(extrae_cantidad(tree))
    parte1 = Moneda+'&from=EUR'
    parte2 = '&amount='+str(Cantidad)

    url = APIUrl + parte1 + parte2
    
    


    payload = {}
    headers= {
      "apikey": '7fDALIQd6ukbS2GrewMNTLNlxqxdyJs7'
    }

    response = requests.request("GET", url, headers=headers, data = payload)

    status_code = response.status_code
    result = response.text
    
    dictionario = json.loads(result)
    monto_final = dictionario['result']
    
    frase_final = str(Cantidad) + ' euros es ' + str(monto_final) + Moneda
    
    return frase_final

In [11]:
mic = sr.Microphone()
instance = sr.Recognizer()
def audio_input():
    mic = sr.Microphone()
    
    with mic as source:
        instance.adjust_for_ambient_noise(source)
        audio = instance.listen(source)
        transcript = instance.recognize_google(audio, language='es-ES', show_all=True)
        
        return transcript['alternative'][0]['transcript']

In [12]:
engine = pyttsx3.init()
def speak(_texto):
    engine.setProperty('voice', 'spanish')
    engine.setProperty('rate', 150)
    engine.say(_texto)
    engine.runAndWait()

In [17]:
def Alexa():
    
    input_text = audio_input()
    
    print(input_text)
    
    frase_speak = convierte_moneda(input_text, _hmm=hmm)

    print(frase_speak)
    speak(frase_speak)
    

In [20]:
Alexa()

dime cuanto es cien euros en BRL
100 euros es 563.3947BRL
